<a href="https://colab.research.google.com/github/Lucasx10/Projetos-Inteligencia-Artificial/blob/main/Algoritmos%20Gen%C3%A9ticos/Alg_genetico_mochila01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução ao problema
 No problema da mochila 0-1, recebemos um conjunto de itens, cada um com um peso e um valor, e precisamos determinar o número de cada item a ser incluído em uma coleção de modo que o peso total seja menor ou igual a um determinado limite e o valor total é o maior possível. Nesta tarefa você deve solucionar este problema utilizando um algoritmo genético.

Para ilustrar este problema, imagine a situação hipotética. Um ladrão entra em uma loja carregando uma mochila (bolsa) que pode carregar 35 kg de peso. A loja possui 10 itens, cada um com peso e preço específicos. Agora, o dilema do ladrão é fazer uma seleção de itens que maximize o valor (ou seja, o preço total) sem exceder o peso da mochila. Temos que ajudar o ladrão a fazer a seleção.

 Utilize os seguintes itens para colocar na mochila:

| Item  |  Peso |  Valor |
| --- | --- | --- | 
| 1 | 3 | 266 |
| 2 | 13 | 442 |
| 3 | 10 | 671 |
| 4 | 9 | 526 |
| 5 | 7 | 388 |
| 6 | 1 | 245 |
| 7 | 8 | 210 |
| 8 | 8 | 145 |
| 9 | 2 | 126 |
| 10 | 9 | 322 |

## **Passo 1: Configuração do ambiente**

Primeiro, vamos configurar o ambiente no Google Colab. Importar as bibliotecas necessárias:

In [36]:
import random
import numpy as np

##**Passo 2: Definição dos dados**
Vamos definir os dados do problema - os pesos e valores dos itens disponíveis para colocar na mochila:

In [37]:
items = [
    (3, 266),
    (13, 442),
    (10, 671),
    (9, 526),
    (7, 388),
    (1, 245),
    (8, 210),
    (8, 145),
    (2, 126),
    (9, 322)
]

peso_max = 35

Cada item é representado por uma tupla com o peso e o valor. O max_peso representa o peso máximo que a mochila pode suportar.

##**Passo 3: Representação do Cromossomo (objetivo)**
Vamos representar uma solução candidata (ou cromossomo) como uma lista de 0s e 1s, onde cada posição representa um item. Um valor de 1 indica que o item está presente na mochila, e um valor de 0 indica que o item está ausente. O tamanho da lista será igual ao número de itens disponíveis. Por exemplo, o cromossomo `[1, 0, 1, 0, 1, 0, 0, 1, 0, 1]` significa que os itens `1, 3, 5, 8 e 10` da lista estão presentes na mochila.

## **Passo 3: Função de Avaliação (fitness function)**
A função de avaliação atribui uma pontuação ao cromossomo com base no valor total dos itens presentes na mochila, considerando também o limite de peso. Quanto maior o valor, melhor é o cromossomo. a função de avaliação definida fica assim:

In [38]:
def funcao_avaliacao(combinacao):
    peso_total = 0
    valor_total = 0
    for i in range(len(combinacao)):
        if combinacao[i] == 1:
            peso_total += items[i][0]
            valor_total += items[i][1]
    if peso_total > peso_max:
        valor_total = 0
    return valor_total

A função percorre o cromossomo e calcula o peso total e o valor total dos itens selecionados. Se o peso total exceder o limite da mochila, o valor total é definido como 0.

##**Passo 5: Operadores Genéticos**
Agora, vamos implementar os operadores genéticos - seleção, cruzamento (crossover) e mutação.
### Seleção
Foi utilizado o método da roleta viciada, onde a probabilidade de seleção de um cromossomo é proporcional à sua pontuação de avaliação. Foi implementada a função de seleção da seguinte forma:

In [39]:
def selecao(populacao, fitness):
    probabilidade = [f / sum(fitness) for f in fitness]
    indices_selecionados = np.random.choice(len(populacao), size=len(populacao), p=probabilidade)
    return [populacao[i] for i in indices_selecionados]

### Crossover
Para o crossover, vamos usar o crossover de um ponto, onde um ponto de corte é escolhido aleatoriamente, e os genes dos pais são trocados para formar os filhos. Implementaremos a função de crossover da seguinte forma:

In [40]:
def crossover(individuo1, individuo2):
    crossover_point = random.randint(1, len(individuo1) - 1)
    child1 = individuo1[:crossover_point] + individuo2[crossover_point:]
    child2 = individuo2[:crossover_point] + individuo1[crossover_point:]
    return child1, child2

### Mutação 
foi invertido aleatoriamente um gene no cromossomo. Implementaremos a função de mutação da seguinte forma:

In [41]:
def mutacao(combinacao, taxa_mutacao):
    for i in range(len(combinacao)):
        if random.random() < taxa_mutacao:
            combinacao[i] = 1 - combinacao[i]
    return combinacao

## **Passo 6: Algoritmo Genético**
Agora, vamos combinar todas as partes e implementar o algoritmo genético:

In [42]:
def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao):
    populacao = []
    for _ in range(tamanho_populacao):
        individuo = [random.randint(0, 1) for _ in range(len(items))]
        populacao.append(individuo)

    for _ in range(geracoes):
        fitness = [funcao_avaliacao(individuo) for individuo in populacao]

        melhor_combinacao = populacao[np.argmax(fitness)]
        best_fitness = funcao_avaliacao(melhor_combinacao)
        if best_fitness == 0:
            break

        populacao_selecionada = selecao(populacao, fitness)

        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            individuo1, individuo2 = random.sample(populacao_selecionada, 2)
            child1, child2 = crossover(individuo1, individuo2)
            nova_populacao.append(mutacao(child1, taxa_mutacao))
            nova_populacao.append(mutacao(child2, taxa_mutacao))

        populacao = nova_populacao

    return melhor_combinacao

##**Passo 7: Executando o Algoritmo Genético**
Agora, podemos executar o algoritmo genético com os parâmetros desejados:

In [92]:
# Parâmetros do algoritmo genético
population_size = 100  # Tamanho da população
generations = 100  # Número de gerações
mutation_rate = 0.08  # Taxa de mutação

# Execução do algoritmo genético
solucao = algoritmo_genetico(population_size, generations, mutation_rate)
print("Melhor solução encontrada:", solucao)

Melhor solução encontrada: [1, 0, 1, 1, 1, 1, 0, 0, 1, 0]


In [93]:
solucao

[1, 0, 1, 1, 1, 1, 0, 0, 1, 0]

In [94]:
funcao_avaliacao(solucao)

2222

O valor total dos items presentes na mochila. 